In [1]:
import mujoco
import numpy as np
import pinocchio as pin

# from robot_descriptions.skydio_x2_description import URDF_PATH
# from robot_descriptions.skydio_x2_mj_description import MJCF_PATH
from robot_descriptions.z1_description import URDF_PATH
from robot_descriptions.z1_mj_description import MJCF_PATH

from mujoco_sysid import parameters, regressors
from mujoco_sysid.utils import muj2pin

In [2]:
mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
# enable energy
mjmodel.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY
# disable friction, contact and limits
mjmodel.opt.disableflags |= (
    mujoco.mjtDisableBit.mjDSBL_CONTACT | mujoco.mjtDisableBit.mjDSBL_FRICTIONLOSS | mujoco.mjtDisableBit.mjDSBL_LIMIT
)
mjdata = mujoco.MjData(mjmodel)

pinmodel = pin.buildModelFromUrdf(URDF_PATH)
pindata = pin.Data(pinmodel)

In [3]:
# np.random.seed(10)

In [4]:
q, v = np.random.randn(mjmodel.nq), np.zeros(mjmodel.nv)


pinq, pinv = muj2pin(q, v)

In [5]:
mjdata.qpos[:] = q.copy()
mjdata.qvel[:] = v.copy()

mujoco.mj_step(mjmodel, mjdata)

mj_en = mjdata.energy.copy()
mj_en[0] += regressors.potential_energy_bias(mjmodel)

mj_en, np.sum(mj_en)

(array([-1.21974604,  0.        ]), -1.2197460404740708)

In [6]:
(
    np.sum(
        [
            pin.computePotentialEnergy(pinmodel, pindata, pinq),
            pin.computeKineticEnergy(pinmodel, pindata, pinq, pinv),
        ]
    ),
    [
        pin.computePotentialEnergy(pinmodel, pindata, pinq),
        pin.computeKineticEnergy(pinmodel, pindata, pinq, pinv),
    ],
)

(-1.2197457679147197, [-1.2197457679147197, 0.0])

In [7]:
theta = np.concatenate([parameters.get_dynamic_parameters(mjmodel, i) for i in mjmodel.jnt_bodyid])

theta.shape, theta[:10]

((60,),
 array([ 6.73326000e-01,  1.66311522e-06, -1.69664685e-04,  1.56021081e-02,
         1.64484975e-03, -5.95801269e-08,  1.08083746e-03, -4.38536417e-07,
         4.43147004e-06,  8.39403034e-04]))

In [8]:
params = []

for i in range(len(pinmodel.inertias) - 1):
    params.extend(pinmodel.inertias[i + 1].toDynamicParameters())

    last_params = np.array(params[-10:])
    last_theta = theta[i * 10 : (i + 1) * 10]

    # mass should match
    assert np.isclose(last_params[0], last_theta[0])

    # lever arm should match
    assert np.allclose(last_params[1:4], last_theta[1:4])

    print(f"for body {i} norm of difference is {np.linalg.norm(last_params - last_theta)}")

params = np.array(params)

params.shape, np.linalg.norm(params - theta)

for body 0 norm of difference is 4.903343079507622e-07
for body 1 norm of difference is 2.5970031568529883e-06
for body 2 norm of difference is 2.650078190181762e-07
for body 3 norm of difference is 3.701542518534774e-07
for body 4 norm of difference is 8.053173890678282e-08
for body 5 norm of difference is 7.002258930267969e-08


((60,), 2.6839308799870235e-06)

In [9]:
reg_en = regressors.mj_energyRegressor(mjmodel, mjdata)[2]

reg_en @ theta

-1.2197460404740712